# Catnip

## Setup

### Colab

In [ ]:
# --- Scripts for Colab ---
# Clone repo
!git clone -b yolo-finetune --recurse-submodules https://github.com/rifusaki/catnip.git
%cd /content/catnip

# Authenticate with Google
from google.colab import auth
auth.authenticate_user()

# Install gcsfuse
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

# Mount bucket
!mkdir -p /content/catnip/data
!gcsfuse --implicit-dirs catnip-data /content/catnip/data

# Install packages not included in Colab
%pip install ultralytics pydantic pydantic-settings omegaconf

import os
from pathlib import Path
os.chdir(str(Path.cwd())+'/catnip')

### Local

In [1]:
# Autoreload for debugging
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path
os.chdir(Path.cwd().parent)

### General

In [2]:
# Dependencies and configuration
from src.config import settings, setup_dirs

print("Working directory set to:", Path.cwd())
izutsumiPaths, notIzutsumiPaths = setup_dirs()

Working directory set to: /Users/rifusaki/repos/catnip


## Pre-processing

### Panel extraction

In [ ]:
from modules.coreMPE.src.adenzu_panel.image_processing import panel


_ = panel.extract_panels_for_images_in_folder_recursive(
    input_dir=str(settings.paths.pages_dir),
    output_dir=str(settings.paths.panels_dir),
    split_joint_panels=False,   # maps to --split-joint-panels
    fallback=True              # maps to --fallback
)

### Head crops

In [ ]:
from src.preprocess.headExtraction import anime_extraction_recursive


valid_exts = {".jpg", ".jpeg", ".png"}
panel_paths = sorted(
    [p for p in settings.paths.panels_dir.iterdir() if p.suffix.lower() in valid_exts]
)
num_crops = anime_extraction_recursive()

print(f"Extracted {num_crops} faces")

## Catnip core

### Dataset preparation

In [ ]:
from src.training.preparation import prepare_data

prepare_data(izutsumiPaths, notIzutsumiPaths)

### Training

In [4]:
# For Apple Silicon
%env PYTORCH_ENABLE_MPS_FALLBACK=1
from ultralytics import YOLO

model = YOLO(settings.paths.model_dir/'yolo11x-cls.pt')
model.info(detailed=True)

env: PYTORCH_ENABLE_MPS_FALLBACK=1
layer                                    name                type  gradient  parameters               shape        mu     sigma
    0                     model.0.conv.weight              Conv2d     False        2592       [96, 3, 3, 3] -0.000392     0.151        float32
    1                       model.0.bn.weight         BatchNorm2d     False          96                [96]      1.42      1.48        float32
    1                         model.0.bn.bias         BatchNorm2d     False          96                [96]    -0.407      2.61        float32
    2                             model.0.act                SiLU     False           0                  []         -         -              -
    3                     model.1.conv.weight              Conv2d     False      165888     [192, 96, 3, 3] -0.000697    0.0306        float32
    4                       model.1.bn.weight         BatchNorm2d     False         192               [192]      4.73     

(176, 29637064, 0, 112.0110592)

In [ ]:
model.train(
        data="config/izutsumiTraining.yaml",
        epochs=200,
        imgsz=settings.params.img_size,
        batch=16,
        lr0=1e-2,       # lower LR for finetuning
        freeze=9,      # freeze backbone layers
        project="runs/izutsumi_finetune",
        name="tl",
        device='cpu', # "mps", "cuda", "cpu"
        workers=8,
        resume=False,
        cache=False     # True for Colab
    )

### Evaluation

In [ ]:
# Evaluate
metrics = model.val()
print(metrics)  

### Prediction

In [ ]:
# Predict on unseen images
model.predict(
    source="data/recognition/izutsumiTraining/val/images",
    save=True,
    conf=0.5
)